## Conectar con la base de datos

In [1]:
# Celda 1: Configuración del entorno
import os
import sys
import logging

# Añadir el directorio raíz al path de Python
project_root = os.path.abspath(os.path.join('..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Configurar logging para ver mensajes
logging.basicConfig(level=logging.INFO)

In [2]:
# Celda 2: Importar el módulo de base de datos
from core.database import Database

In [3]:
# Celda 3: Probar la conexión básica
Database.initialize()
try:
    with Database.get_cursor() as cursor:
        # Consulta simple para verificar la conexión
        cursor.execute("SELECT version()")
        db_version = cursor.fetchone()
        print("✅ Conexión exitosa a PostgreSQL")
        print(f"Versión: {db_version[0]}")
        
except Exception as e:
    print(f"❌ Error al conectar: {e}")

INFO:core.database:Intentando conectar a la base de datos con config: {'host': 'localhost', 'port': '5432', 'database': 'godonto_db1', 'user': 'godonto_user1', 'password': '27934140'}
INFO:core.database:Pool de conexiones a la base de datos inicializado correctamente
INFO:core.database:Conexión a la base de datos verificada correctamente


✅ Conexión exitosa a PostgreSQL
Versión: PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit


In [1]:
! pip install pandas

  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 1.3 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/11.5 MB 1.3 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.5 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.5 MB 1.2 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.5 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.5 MB 1.3 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.5 MB 1.1 MB/s eta 0:00:09
   ------- -----------------------------

In [4]:
# Celda 4: Importar pandas y configurar visualización
import pandas as pd

# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [5]:
# Celda 5: Función auxiliar para ejecutar consultas y mostrar resultados como DataFrame
def query_to_dataframe(query, params=None):
    try:
        with Database.get_cursor() as cursor:
            cursor.execute(query, params or ())
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            return pd.DataFrame(data, columns=columns)
    except Exception as e:
        print(f"Error al ejecutar consulta: {e}")
        return pd.DataFrame()

In [6]:
# Celda 6: Obtener lista de tablas en la base de datos
tables_query = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public' 
ORDER BY table_name
"""

tables_df = query_to_dataframe(tables_query)
print("Tablas disponibles en la base de datos:")
display(tables_df)

Tablas disponibles en la base de datos:


,table_name
0,access_keys
1,appointment_treatments
2,appointments
3,clients
4,debt_payments
5,debts
6,dentists
7,medical_history
8,payments
9,quote_treatments


In [8]:
# Celda 7: Visualizar datos de una tabla específica (ejemplo con una tabla llamada 'clientes')
# Reemplaza 'clientes' con el nombre de tu tabla real

tabla = int(input("Introduce el numero de la tabla que deseas visualizar: ")) or tables_df['table_name'].iloc[0]

if not tables_df.empty:
    sample_table = tables_df['table_name'].iloc[tabla]  # Toma la primera tabla como ejemplo
    print(f"\nMostrando contenido de la tabla: {sample_table}")
    
    data_query = f"SELECT * FROM {sample_table} LIMIT 10"
    table_data = query_to_dataframe(data_query)
    display(table_data)
else:
    print("No se encontraron tablas en la base de datos.")


Mostrando contenido de la tabla: debt_payments


,id,payment_id,debt_id,amount_applied,created_at


In [30]:
# Celda 8: Visualizar estructura de una tabla específica
if not tables_df.empty:
    sample_table = tables_df['table_name'].iloc[tabla]  # Toma la primera tabla como ejemplo
    print(f"\nEstructura de la tabla: {sample_table}")
    
    structure_query = """
    SELECT column_name, data_type, is_nullable, column_default 
    FROM information_schema.columns 
    WHERE table_name = %s
    """
    table_structure = query_to_dataframe(structure_query, (sample_table,))
    display(table_structure)


Estructura de la tabla: treatments


,column_name,data_type,is_nullable,column_default
0,updated_at,timestamp without time zone,NO,CURRENT_TIMESTAMP
1,created_at,timestamp without time zone,NO,CURRENT_TIMESTAMP
2,id,integer,NO,nextval('treatments_id_seq'::regclass)
3,price,numeric,NO,None
4,duration,interval,NO,None
5,is_active,boolean,NO,true
6,name,character varying,NO,None
7,description,text,YES,None


In [31]:
# Celda 5: Cerrar conexiones
Database.close_all_connections()
print("Conexiones cerradas")

INFO:core.database:Todas las conexiones de la base de datos cerradas


Conexiones cerradas
